# Genereer A4's om uit te delen

In [1]:
students_file = 'students.xlsx'
password_secret = 'cb8116ae8acd84126cb3316ff201fc755993e026b74df8849e05456513d52378'

In [2]:
md = '''# <h1 style="margin-bottom: 4px;">%s</h1>
<h2 class="normal muted">Notebook examen</h2>

## <span class="normal">Je gebruikersnaam:</span> %s
## <span class="normal">Je wachtwoord:</span> %s

### Introductie

We willen de volgende toets gaan uitvoeren op een Jupyter Notebook, net als jullie steeds in het huiswerk doen.
Dit is nog nooit gebeurd aan de UvA. Vandaar dat we nu graag met jullie testen of het werkt.
Ga naar **<http://notebookexam.com>**, log in met je uvanetid en het wachtwoord hierboven en druk op "start tentamen".

Een notebook omgeving opent op een computer die alleen door jou gebruikt wordt. Je kunt hier alles doen wat je gewend bent. 
Ga naar de map "Bonus 21 november", open het notebook "Tentamen.ipynb" en maak de vragen.  

Je notebook wordt bewaard onder jouw naam, en je krijgt een bonus als je  de vragen goed hebt. In een kwartiertje ben je klaar.

### Afsluiten en inleveren

Doorloop de volgende stappen:

1. Herstart je kernel.
2. Run all cells
3. Kijk of alles er nog zo mooi uit zag als daarvoor
	* Repareer eventueel een foutje
4. Bewaar je notebook
5. Lever je notebook in door op **Beëindig tentamen** te drukken.

<hr>

Veel succes en bedankt voor het meedoen.'''

In [3]:
from md2pdf.core import md2pdf
from hashauthenticator import generate_password_digest
import pandas as pd

def generate_password(username):
    return generate_password_digest(username, password_secret)[:6]

### Maak lijst van gebruikers en wachtwoorden

In [4]:
students = pd.read_excel(students_file)
students['UvAnetID'] = students['UvAnetID'].apply(str)

def student_name(student):
    if len(student['MiddleName'].strip()):
        return student['FirstName'].strip() +' '+ student['MiddleName'].strip() +' '+ student['LastName'].strip()
    else:
        return student['FirstName'].strip() +' '+ student['LastName'].strip()

users = [ (student_name(student), \
           student['UvAnetID'], \
           generate_password(student['UvAnetID'])) for index, student in students.iterrows() ]

### Genereer PDF's op basis van Markdown en voeg samen

In [ ]:
!mkdir student-sheets

from PyPDF2 import PdfFileMerger
merger = PdfFileMerger()

users = sorted(users, key=lambda user: user[0])  # dit sorteert op voornaam in deze setup

for i, user in enumerate(users):
    md2pdf('student-sheets/%s.pdf' % user[1], md_content=(md) % user, css_file_path='md-style.css')
    
    merger.append(open('student-sheets/%s.pdf' % user[1], 'rb'))
        
with open('all-students.pdf', 'wb') as f:
    merger.write(f)